# TESTING

In [ ]:
import os
import json
from newsapi.newsapi_client import NewsApiClient
import requests
import re
from datetime import datetime, timedelta, date
from bs4 import BeautifulSoup
import html


In [ ]:
client = NewsApiClient('ee9ebbc8037b49a58370ee1446c8f8db')
response = client.get_top_headlines(language='de', sources='die-zeit')
if response['status'] == 'ok':
    articles = response['articles']

    print(json.dumps(articles[0], indent=2))


    article_url = articles[1]['url']
    print("article_url:", article_url)


In [ ]:
# get the article from the url
def get_complete_article(url:str):
    article_response = requests.get(url)
    if article_response.status_code != 200:
        print(f"Failed to retrieve article. Status code: {article_response.status_code}")
        return
    # print(article_response.text)
    for line in article_response.text.split('\n'):
        linestart = "\"articleBody\": "
        if line.strip().startswith(linestart):
            line = line.strip(linestart).split("©")[0]  # ignore dpa and other sources in string
            return html.unescape(line)  # &#34; -> "
    print("Content not found in article")
    return 

    
print("Article 1:")
print(get_complete_article("https://www.zeit.de/news/2024-12/03/schuesse-in-berlin-gesundbrunnen-ein-mann-schwer-verletzt"))
print("\n")

print("Article 2:")
print(get_complete_article("https://www.zeit.de/news/2024-12/03/sperren-beschaedigt-weihnachtmarkt-bleibt-geschlossen"))

# Klasse, die so eig übernommen werden kann

In [19]:
import json
from newsapi.newsapi_client import NewsApiClient
import requests
import html


class Newspaper:
    def __init__(self, api_key:str):
        self.client = NewsApiClient(api_key=api_key)
        self.articles = None
        self.source = 'die-zeit,focus,bild'

    @property
    def amount_articles(self):
        return len(self.articles)

    def get_articles(self):
        '''return and refresh the articles with the top headlines'''
        response = self.client.get_top_headlines(language='de', sources=self.source)
        if response['status'] == 'ok':
            self.articles = response['articles']
            for article in self.articles:
                article['title'] = html.unescape(article['title'])
        return self.articles
    

    @staticmethod
    def get_complete_article(url:str):
        article_response = requests.get(url)
        if article_response.status_code != 200:
            print(f"Failed to retrieve article. Status code: {article_response.status_code}")
            return
        # print(article_response.text)
        for line in article_response.text.split('\n'):
            linestart = "\"articleBody\": "
            if line.strip().startswith(linestart):
                line = line.strip(linestart).split("©")[0]  # ignore dpa and other sources in string
                return html.unescape(line)  # &#34; -> "
        return "Entschuldigung, der Artikel konnte nicht gefunden werden."
    

# Example
newsp = Newspaper('ee9ebbc8037b49a58370ee1446c8f8db')
print("First 2 articles: ")
print(json.dumps(newsp.get_articles(), indent=2, ensure_ascii=False))
print("Article 1:")
print(Newspaper.get_complete_article(newsp.articles[0]['url']))
print("\n")
print("Article 2:")
print(Newspaper.get_complete_article(newsp.articles[1]['url']))


First 2 articles: 
[
  {
    "source": {
      "id": "die-zeit",
      "name": "Die Zeit"
    },
    "author": "dpa",
    "title": "Kriminalität: Mann nach Schusswechsel in Kiel festgenommen",
    "description": "Hier finden Sie Informationen zu dem Thema „Kriminalität“. Lesen Sie jetzt „Mann nach Schusswechsel in Kiel festgenommen“.",
    "url": "https://www.zeit.de/news/2024-12/03/ein-mann-nach-schusswechsel-in-kiel-festgenommen",
    "urlToImage": "https://img.zeit.de/news/2024-12/03/ein-mann-nach-schusswechsel-in-kiel-festgenommen-image-group/wide__1300x731",
    "publishedAt": "2024-12-03T17:30:05+00:00",
    "content": "In Kiel ist ein 38-Jähriger bei einem Schusswechsel mit der Polizei schwer verletzt worden. Der Mann habe im Stadtteil Ellerbek am Mittag auf Polizeikräfte geschossen, weshalb auch die Beamten ihre S… [+1188 chars]"
  },
  {
    "source": {
      "id": "die-zeit",
      "name": "Die Zeit"
    },
    "author": "Konstantin Zimmermann, Felix Lill",
    "title": "Südk